In [ ]:
import pandas as pd
import numpy as np

hex_salmon = '#F68F83'
hex_gold = '#BC9661'
hex_indigo = '#2D2E5F'
hex_maroon = '#8C4750'
hex_white = '#FAFAFA'
hex_blue = '#7EB5D2'

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as dates

import matplotlib.font_manager as font_manager
mpl.font_manager._rebuild()

mpl.rcParams['font.family'] = 'SF Mono'
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['axes.titleweight'] = 'semibold'
mpl.rcParams['axes.labelweight'] = 'medium'
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[hex_indigo, hex_salmon, hex_maroon])
mpl.rcParams["figure.titlesize"] = 'large'
mpl.rcParams["figure.titleweight"] = 'semibold'

from termcolor import colored

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, accuracy_score

import tensorflow as tf

from entsoe import EntsoePandasClient

In [ ]:
# # multivariate multi-step encoder-decoder lstm
# from math import sqrt
# from numpy import split
# from numpy import array
# from pandas import read_csv
# from sklearn.metrics import mean_squared_error
# from matplotlib import pyplot
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Flatten
# from keras.layers import LSTM
# from keras.layers import RepeatVector
# from keras.layers import TimeDistributed

# # load all data
# dataset = read_csv('household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])

# # mark all missing values
# dataset.replace('?', 'nan', inplace=True)
# # make dataset numeric
# dataset = dataset.astype('float32')

# # fill missing values with a value at the same time one day ago
# def fill_missing(values):
# 	one_day = 60 * 24
# 	for row in range(values.shape[0]):
# 		for col in range(values.shape[1]):
# 			if isnan(values[row, col]):
# 				values[row, col] = values[row - one_day, col]

# # fill missing
# fill_missing(dataset.values)

# # add a column for for the remainder of sub metering
# values = dataset.values
# dataset['sub_metering_4'] = (values[:,0] * 1000 / 60) - (values[:,4] + values[:,5] + values[:,6])

# # save updated dataset
# dataset.to_csv('household_power_consumption.csv')

In [ ]:
# # multivariate multi-step encoder-decoder lstm
# from math import sqrt
# from numpy import split
# from numpy import array
# from pandas import read_csv
# from sklearn.metrics import mean_squared_error
# from matplotlib import pyplot
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Flatten
# from keras.layers import LSTM
# from keras.layers import RepeatVector
# from keras.layers import TimeDistributed

# import pandas as pd

# # split a univariate dataset into train/test sets
# def split_dataset(data):
# 	# split into standard weeks
# 	train, test = data[1:-328], data[-328:-6]
# 	# restructure into windows of weekly data
# 	train = array(split(train, len(train)/7))
# 	test = array(split(test, len(test)/7))
# 	return train, test

# # evaluate one or more weekly forecasts against expected values
# def evaluate_forecasts(actual, predicted):
# 	scores = list()
# 	# calculate an RMSE score for each day
# 	for i in range(actual.shape[1]):
# 		# calculate mse
# 		mse = mean_squared_error(actual[:, i], predicted[:, i])
# 		# calculate rmse
# 		rmse = sqrt(mse)
# 		# store
# 		scores.append(rmse)
# 	# calculate overall RMSE
# 	s = 0
# 	for row in range(actual.shape[0]):
# 		for col in range(actual.shape[1]):
# 			s += (actual[row, col] - predicted[row, col])**2
# 	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
# 	return score, scores

# # summarize scores
# def summarize_scores(name, score, scores):
# 	s_scores = ', '.join(['%.1f' % s for s in scores])
# 	print('%s: [%.3f] %s' % (name, score, s_scores))

# # convert history into inputs and outputs
# def to_supervised(train, n_input, n_out=7):
# 	# flatten data
# 	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
# 	X, y = list(), list()
# 	in_start = 0
# 	# step over the entire history one time step at a time
# 	for _ in range(len(data)):
# 		# define the end of the input sequence
# 		in_end = in_start + n_input
# 		out_end = in_end + n_out
# 		# ensure we have enough data for this instance
# 		if out_end <= len(data):
# 			X.append(data[in_start:in_end, :])
# 			y.append(data[in_end:out_end, 0])
# 		# move along one time step
# 		in_start += 1
# 	return array(X), array(y)

# # train the model
# def build_model(train, n_input):
# 	# prepare data
# 	train_x, train_y = to_supervised(train, n_input)
# 	# define parameters
# 	verbose, epochs, batch_size = 5, 2, 16
# 	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
# 	# reshape output into [samples, timesteps, features]
# 	train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
# 	# define model
# 	model = Sequential()
# 	model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
# 	model.add(RepeatVector(n_outputs))
# 	model.add(LSTM(200, activation='relu', return_sequences=True))
# 	model.add(TimeDistributed(Dense(100, activation='relu')))
# 	model.add(TimeDistributed(Dense(1)))
# 	model.compile(loss='mse', optimizer='adam')
# 	# fit network
# 	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
# 	return model

# # make a forecast
# def forecast(model, history, n_input):
# 	# flatten data
# 	data = array(history)
# 	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
# 	# retrieve last observations for input data
# 	input_x = data[-n_input:, :]
# 	# reshape into [1, n_input, n]
# 	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
# 	# forecast the next week
# 	yhat = model.predict(input_x, verbose=0)
# 	# we only want the vector forecast
# 	yhat = yhat[0]
# 	return yhat

# # evaluate a single model
# def evaluate_model(train, test, n_input):
# 	# fit model
# 	model = build_model(train, n_input)
# 	# history is a list of weekly data
# 	history = [x for x in train]
# 	# walk-forward validation over each week
# 	predictions = list()
# 	for i in range(len(test)):
# 		# predict the week
# 		yhat_sequence = forecast(model, history, n_input)
# 		# store the predictions
# 		predictions.append(yhat_sequence)
# 		# get real observation and add to history for predicting the next week
# 		history.append(test[i, :])
# 	# evaluate predictions days for each week
# 	predictions = array(predictions)
# 	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
# 	return score, scores, model


In [6]:
# multivariate multi-step encoder-decoder lstm
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed

import pandas as pd

# split a univariate dataset into train/test sets
def split_dataset(data):
	# split into standard weeks
	train, test = data[data.index.year < 2018], data[data.index.year == 2018]
	# restructure into windows of weekly data
	# train = array(split(train, len(train)/7))
	# test = array(split(test, len(test)/7))
	return train, test

# evaluate one or more weekly forecasts against expected values
def evaluate_forecasts(actual, predicted):
	scores = list()
	# calculate an RMSE score for each day
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

# summarize scores
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=7):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)

# train the model
def build_model(train, n_input):
	# prepare data
	train_x, train_y = to_supervised(train, n_input)
	# define parameters
	verbose, epochs, batch_size = 5, 2, 16
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# reshape output into [samples, timesteps, features]
	train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
	# define model
	model = Sequential()
	model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
	model.add(RepeatVector(n_outputs))
	model.add(LSTM(200, activation='relu', return_sequences=True))
	model.add(TimeDistributed(Dense(100, activation='relu')))
	model.add(TimeDistributed(Dense(1)))
	model.compile(loss='mse', optimizer='adam')
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model

# make a forecast
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, :]
	# reshape into [1, n_input, n]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat

# evaluate a single model
def evaluate_model(train, test, n_input):
	# fit model
	model = build_model(train, n_input)
	# history is a list of weekly data
	history = [x for x in train]
	# walk-forward validation over each week
	predictions = list()
	for i in range(len(test)):
		# predict the week
		yhat_sequence = forecast(model, history, n_input)
		# store the predictions
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
		history.append(test[i, :])
	# evaluate predictions days for each week
	predictions = array(predictions)
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores, model


In [ ]:

# # load the new file
# # dataset = read_csv('household_power_consumption_days.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])
# dataset = features = pd.read_pickle(f"./features.pkl")

# # split into train and test
# train, test = split_dataset(dataset.values)
# # evaluate model and get scores
# n_input = 14
# score, scores = evaluate_model(train, test, n_input)
# # summarize scores
# summarize_scores('lstm', score, scores)
# # plot scores
# days = ['sun', 'mon', 'tue', 'wed', 'thr', 'fri', 'sat']
# pyplot.plot(days, scores, marker='o', label='lstm')
# pyplot.show()

In [4]:
dataset = features = pd.read_pickle(f"./features.pkl")

In [7]:
train, test = split_dataset(dataset)

In [ ]:
n_input = 7
score, scores, model = evaluate_model(train, test, n_input)

In [ ]:
prediction = model.predict(test)

In [ ]:
actual_pred2 = np.hstack(prediction[:,0])

In [ ]:
####################################################################################################
# Pre

actual_pred = actual_pred2

# actual_pred = pd.DataFrame()

# actual_pred['Actual'] = features['ID3'][0:len(prediction)]
# actual_pred['Predicted'] = prediction[:, 0]

####################################################################################################
# Plot 1

fig, ax = plt.subplots(figsize = (10 ,5))

ax.plot(actual_pred2)

####################################################################################################
# # Plot 2

# x, y = actual_pred.index, actual_pred['Predicted']

# ax.plot(x, y, alpha = 0.5)

# plt.xticks(rotation = 90);

In [ ]:
# summarize scores
summarize_scores('lstm', score, scores)
# plot scores
days = ['sun', 'mon', 'tue', 'wed', 'thr', 'fri', 'sat']
pyplot.plot(days, scores, marker='o', label='lstm')
pyplot.show()